Draw a Sankey diagram!

Load the data created by the script `scripts/assemble_flows.py`:

In [ ]:
import pandas as pd
flows = pd.read_csv('data/flows.csv').query('year == "2016"')

Define the Sankey diagram structure and draw:

In [ ]:
from floweaver import *

sectors = ['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15a', '15b', '16', '17', '18', '19', '20']

semis = ['Hot rolled', 'Plate', 'Cold rolled', 'Hot dipped galvanised',
       'Electro coated', 'Organic coated', 'Tin plate', 'Tubes and pipes',
       'Railway track material', 'Sheet piling and rolled accessories',
        'Heavy sections', 'Light sections',
       'Hot rolled bars in lengths', 'Bright bars', 'Reinforcing bar', 'Rods']

products = [('Structural', ['Structural steelwork and building and civil engineering']),
       'Mechanical engineering', 'Vehicles',
       'Electrical engineering', 'Shipbuilding',
       'Boilers, drums and other vessels','Metal goods', 
       'Other industries']

psemi = Partition.Simple('material', semis)
pprod = Partition.Simple('material', sectors)

pmat = Partition.Simple('material', [
    ('Flat', ['Hot rolled', 'Plate', 'Cold rolled', 'Hot dipped galvanised',
              'Electro coated', 'Organic coated', 'Tin plate', 'Tubes and pipes']),
    ('Long', ['Railway track material', 'Sheet piling and rolled accessories', 'Heavy sections', 'Light sections',
              'Hot rolled bars in lengths', 'Bright bars', 'Reinforcing bar', 'Rods']),
] + sectors)

nodes = {
    'prod': ProcessGroup(['uk_production'], title='UK production'),
    'semi': ProcessGroup(['uk_intermediate'], psemi, title='Semi'),
    'manuf': ProcessGroup(sectors, Partition.Simple('process', sectors), title='Manufacturing'),
    'products': ProcessGroup(['products'], pprod, title='Products'),
    'demand': ProcessGroup(['uk_demand'], pprod, title='UK demand'),
    
    'scrap': ProcessGroup(['scrap'], title='Manufacturing scrap'),
    
    'semi_export': ProcessGroup(['exports'], title='Export of semis'),
    'semi_import': ProcessGroup(['imports'], title='Import'),
    'comp_import': ProcessGroup(['component_imports'], title='Import components'),
    'prod_export': ProcessGroup(['product_exports'], title='Export of goods'),
    'prod_import': ProcessGroup(['product_imports'], title='Import of steel-containing goods'),
    'imbalance': ProcessGroup(['imbalance'])
}

ordering = [
    [['semi_import'], ['prod']],
    [['semi_export', 'comp_import'], ['semi']],
    [['imbalance'], ['manuf']],
    [['prod_import'], ['products'], ['scrap']],
    [['prod_export'], ['demand']],
]

bundles = [
    Bundle('prod', 'semi'),
    Bundle('prod', 'semi_export'),
    Bundle('semi_import', 'semi'),
    Bundle('semi', 'manuf'),
    
    Bundle('comp_import', 'manuf'),
    
    Bundle('manuf', 'products'),
    Bundle('manuf', 'scrap'),
    Bundle('manuf', 'prod_export'),
    Bundle('prod_import', 'demand'),
    Bundle('products', 'demand'),
    Bundle('products', 'prod_export'),
    
    Bundle('imbalance', 'products'),
]

sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=pmat)
weave(sdd, flows).to_widget(width=1200, height=1000, debugging=False, align_link_types=False) \
.auto_save_svg('figures/sankey_2016.svg')